#Test with new example

###Process data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os

####Read data

In [ ]:
# sắp xếp tên thư mục giống trên window
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
#Read data test
path = "/content/drive/MyDrive/Project/test"
os.chdir(path)
sort_listdir = sorted_alphanumeric(os.listdir())
print(len(sort_listdir))
data_vungtau = []
for i in sort_listdir:
  with open(path+"/"+i+"/info.txt", 'rb') as f:
      clean_lines = (line.replace(b',,',b',') for line in f)
      content = np.genfromtxt(clean_lines, delimiter=',')
      data_vungtau.append(content)
print("Vung Tau")
data_vungtau[0]

####Filter data

In [ ]:
def linear_equation(x1,y1,x2,y2):
    a = (y1-y2)/(x1-x2)
    c = y1-a*x1
    b = -1
    return a, b, c
    
def safe(value1, value2):
    if value1*value2>=0:
        return True
    return False

In [ ]:
# Các điểm cho trước trên vùng lọc
x8 = 0.0
y8 = 1130.0
x9 = 807.0
y9 = 606.0
x10 = 2361
y10 = 658
# Điểm để xét trong vùng đang quan sát
X = 807.0
Y = 1130.0

# Đường thẳng lọc data vũng tàu
a6, b6, c6 = linear_equation(x8,y8,x9,y9)
a7, b7, c7 = linear_equation(x9,y9,x10,y10)

In [ ]:
filtered_data_vungtau = []

for datas in data_vungtau:
    filters = []
    for data in datas:
        if (safe(a6*X+b6*Y+c6, a6*((data[4]+data[3])/2)+b6*((data[6]+data[5])/2)+c6)==True 
        and safe(a7*X+b7*Y+c7, a7*((data[4]+data[3])/2)+b7*((data[6]+data[5])/2)+c7)==True):
            filters.append(data)
    filtered_data_vungtau.append(filters)

In [ ]:
len(data_vungtau[0])

2217

In [ ]:
len(filtered_data_vungtau[0])

133

####Tách theo id

In [ ]:
#Ghi file
#hàm ghi file
def ghifile(path, content):
  with open(path+'id'+str(content[0][1])+'.txt', 'w') as f:
    for arr in content:
      arr = arr.tolist()  # conver to Python list
      f.write(','.join(map(str, arr)) + '\n')
  f.close()

In [ ]:
path = "/content/drive/MyDrive/Project/test"
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['vungtau_17.0.avi_save']

In [ ]:
id = []
sequence_id = []
labels = []
for i in filtered_data_vungtau: #i la bo du lieu folder thu i
    for j in i:
      id.append(j[1])
    break
print(list(set(id)))

number_folder = 0
for i in filtered_data_vungtau: #i la bo du lieu folder thu i
    #lấy ra tập chứa các id trong folder
    for j in i:
      id.append(j[1])
    for l in id: #duyet id tu tap id
      for k in data_vungtau[number_folder]: #duyet qua cac frame
        if k[1]==l: #neu id frame = id trong tap id
          sequence_id.append(k[:]) #lay ra thong tin sequence  
      #print(sequence_id)
      #ghi ra file sequence of id
      path = '/content/drive/MyDrive/Project/test/' + sort_listdir[number_folder] + '/'
      ghifile(path, sequence_id)
      sequence_id = [] #khoi tao lai sequence_id = rong
    number_folder = number_folder + 1
    id = [] #khởi tạo lại id = rỗng

[70.0, 8.0, 9.0, 40.0, 79.0, 25.0, 62.0]


In [ ]:
#xoa file info goc
path = '/content/drive/MyDrive/Project/test/'
os.chdir(path)
os.listdir()
for i in os.listdir():
  path = path + i
  os.chdir(path)
  os.listdir()
  !rm info.txt
  path = '/content/drive/MyDrive/Project/test/'

####Tạo dataloader

In [32]:
import os
import math
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

class MyCustomDataset(Sequence):
    def __init__(self, info_files, batch_size=1, max_len=147):
        """Instaniate a custom dataset.

        Attributes
          info_files (list or str): List of info files or only one file. Each file
            should have pattern like 'DendoX/*/info.txt' or 'DenxanhX/*/info.txt'.
          batch_size (int): Batch size.
          max_len (int): Maximum of length of sequence.
        """
        self.batch_size = batch_size
        self.max_len = max_len
        if isinstance(info_files, str):
            info_files = [info_files]
        
        for path in info_files:
            assert os.path.isfile(path), f'Not found info file: {path}'
        self.data = self._load_data(info_files)
    
    def on_epoch_end(self):
        np.random.shuffle(self.data)

    def _load_data(self, file_paths):
        data = []
        for path in file_paths:
            # Load label from file path.
           
            # Store object moving states in a list. So this should be a list of
            # 4-value list.
            sequences = []
            central_list = []
            bbox_list = []
            with open(path) as f:
                for line in f:
                    line = line.strip()
                    row = line.split(',')
                    #row = list(map(float, row))  # convert to float
                    bboxes = row[-4:]
                    #store coordinate central point
                    central = []
                    central.append((float(bboxes[0])+float(bboxes[1]))/2)
                    central.append((float(bboxes[2])+float(bboxes[3]))/2)  
                    #normalize bbox
                    """
                    bboxes_normalize = []
                    bboxes_normalize.append(float(bboxes[0])/2560)
                    bboxes_normalize.append(float(bboxes[1])/1920)
                    bboxes_normalize.append(float(bboxes[2])/2560)
                    bboxes_normalize.append(float(bboxes[3])/1920)
                    """
                    sequences.append(bboxes)
                    central_list.append(central)
                    bbox_list.append(bboxes)
            #process label
            name_province = path.split('/')[-2]
            check = 0
            if 'hanam' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 797:
                    check += 1
            if 'thainguyen' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 839:
                    check += 1
            if 'vungtau' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 727:
                    check += 1
            if (check >= 1): 
              label = 1 #vuot den do
            if (check == 0):
              label = 0 #khong vuot
            #padding 
            #sequences_pad = keras.preprocessing.sequence.pad_sequences([sequences], maxlen=147, padding="pre")
            while len(sequences) < 147:
              sequences.insert(0, [0,0,0,0])

            sequences_pad = sequences[:self.max_len]
            data.append((sequences_pad, label))
        return data
    
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)
    
    def __getitem__(self, index):
        """Returns a batch of shape (N, S, 4)"""
        batch_data = self.data[index:self.batch_size:(index+1)*self.batch_size]
        X_batch = np.empty((self.batch_size, self.max_len, 4))
        y_batch = []
        for sequences, label in batch_data:
            X_batch[:len(sequences), :, :] =  np.array(sequences, dtype='float32')
            y_batch.append(label)
        X_batch = np.array(X_batch, dtype='float32')
        y_batch = np.array(y_batch, dtype='int32')
        return X_batch, y_batch

In [33]:
path = '/content/drive/MyDrive/Project/test/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['vungtau_17.0.avi_save']

In [34]:
dataset = []
label = []
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)


In [35]:
data = np.concatenate(dataset, axis=0)
#label_data = np.concatenate(label, axis=0)

In [36]:
#Read data test
path = "/content/drive/MyDrive/Project/test/vungtau_17.0.avi_save"
os.chdir(path)
sort_listdir = sorted_alphanumeric(os.listdir())
print(len(sort_listdir))
info_data = []
for i in sort_listdir:
  with open(path+"/"+i, 'rb') as f:
      clean_lines = (line.replace(b',,',b',') for line in f)
      content = np.genfromtxt(clean_lines, delimiter=',')
      info_data.append(content)
print("Read Successful!")
info_data[0]

7
Read Successful!


array([[   2.   ,    8.   ,    0.   ,  983.238, 1169.25 ,  904.059,
        1343.35 ],
       [   3.   ,    8.   ,    0.   , 1014.01 , 1187.56 ,  860.527,
        1280.06 ],
       [   4.   ,    8.   ,    0.   , 1043.64 , 1216.56 ,  797.035,
        1201.24 ],
       [   5.   ,    8.   ,    0.   , 1070.72 , 1229.89 ,  761.051,
        1149.62 ],
       [   6.   ,    8.   ,    0.   , 1086.5  , 1241.42 ,  726.282,
        1105.79 ],
       [   7.   ,    8.   ,    0.   , 1107.24 , 1264.52 ,  680.109,
        1046.56 ],
       [   8.   ,    8.   ,    0.   , 1124.46 , 1274.14 ,  654.145,
        1003.74 ],
       [   9.   ,    8.   ,    0.   , 1140.23 , 1282.22 ,  614.092,
         953.033],
       [  10.   ,    8.   ,    0.   , 1148.96 , 1290.43 ,  593.46 ,
         917.492],
       [  11.   ,    8.   ,    0.   , 1158.14 , 1298.41 ,  568.25 ,
         886.753],
       [  12.   ,    8.   ,    0.   , 1165.88 , 1302.84 ,  539.14 ,
         849.999],
       [  13.   ,    8.   ,    0.   , 1181.

In [37]:
data[0]

array([[   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   ,    0.   ],
       [   0.   ,   

####Load model

In [12]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score

In [13]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt

%matplotlib inline

In [14]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

####Chuẩn hóa data

In [38]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import numpy as np

class MinMaxScaler3D(MinMaxScaler):

    def fit_transform(self, X, y=None):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)

In [39]:
scaler = MinMaxScaler3D()
x_scale = scaler.fit_transform(data)

In [17]:
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate
from tensorflow.keras.layers import Layer, LeakyReLU


class Attention(Layer):

    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)

    def __call__(self, inputs):
        """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        """
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
        score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
        score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
        attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
        pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
        attention_vector = Dense(self.units, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, LeakyReLU
from tensorflow.keras.utils import plot_model

model = Sequential([              
      LSTM(units=100,input_shape=(147, 4), return_sequences=True),
      Dropout(0.2),
      LSTM(units=100,return_sequences=True),
      LSTM(units=100,return_sequences=True),
      Dropout(0.2),
      Attention(),
      Dropout(0.2),
      Dense(1, activation='sigmoid'),
])

#Checkpoint
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Project/model.h5',
                             monitor="val_f1_m",
                             verbose=1,
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]
METRICS = [
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    f1_m,
]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS,)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 147, 100)          42000     
_________________________________________________________________
dropout (Dropout)            (None, 147, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 147, 100)          80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 147, 100)          80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 147, 100)          0         
_________________________________________________________________
last_hidden_state (Lambda)   (None, 100)               0         
_________________________________________________________________
attention_score_vec (Dense)  (None, 147, 100)          1

In [19]:
model.load_weights('/content/drive/MyDrive/Project/model.h5')

In [40]:
#predict
y_pred = model.predict(x_scale)

threshold = 0.5
y_pred_label = []
for i in y_pred:
  if i < 0.5:
    y_pred_label.append(0)
  else:
    y_pred_label.append(1)
y_pred = np.array(y_pred_label)
y_pred

array([1, 1, 1, 1, 1, 1, 0])

###Xác định xe vượt, ko vượt

In [41]:
sort_listdir

['id8.0.txt',
 'id9.0.txt',
 'id25.0.txt',
 'id40.0.txt',
 'id62.0.txt',
 'id70.0.txt',
 'id79.0.txt']

In [ ]:
id_vuot=[]
id_kovuot=[]
num = 0
for label_pred in y_pred:
  if label_pred == 1:
    id_vuot.append(sort_listdir[num].strip("id,.txt"))
  else:
    id_kovuot.append(sort_listdir[num].strip("id,.txt"))
  num += 1  

In [ ]:
id_vuot

['8.0', '9.0', '25.0', '40.0', '62.0', '70.0']

In [ ]:
id_kovuot

['79.0']